In [3]:
!pip install langchain langchain_community langchain_groq chromadb cohere

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.9/168.9 kB 2.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 2.7 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 2.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 2.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 2.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.2/82.2 kB 2.5 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1


In [5]:
from langchain_community.vectorstores.chroma import Chroma
from langchain_community.embeddings.ollama import OllamaEmbeddings
from langchain_community.embeddings.cohere import CohereEmbeddings

# embeddings = OllamaEmbeddings(model="mxbai-embed-large")
embeddings = CohereEmbeddings(model="embed-english-v3.0", cohere_api_key='thd6PTeuf6WTw3jNa7DnivLzEnFH7xdVbAStAAiM')
chroma = Chroma(embedding_function=embeddings)


In [6]:
import os
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader

text_splitter = RecursiveCharacterTextSplitter(separators=["\nclass ","\ndef ","\n    def"], chunk_size=1000, strip_whitespace=False)

for root, dirs, files in os.walk('fastapi_solo'):
    current_dir = os.path.basename(root)
    if current_dir.startswith('__') or current_dir.startswith('.'):
        continue
    for file in files:
        if not file.endswith('.py') or file.startswith('__'):
            continue
        full_path = os.path.join(root, file)
        print(f'Loading {full_path}')
        docs = TextLoader(full_path).load()
        chunks = text_splitter.split_documents(docs)
        # print("-------------------------------------".join([chunk.page_content for chunk in chunks]))
        chroma.add_documents(chunks)
            

Loading fastapi_solo/dependencies.py
Loading fastapi_solo/serialization/schema_models.py
Loading fastapi_solo/serialization/schemas.py
Loading fastapi_solo/aio/database.py
Loading fastapi_solo/aio/inject.py
Loading fastapi_solo/aio/utils.py
Loading fastapi_solo/aio/monkeypatch.py
Loading fastapi_solo/aio/solipsist.py
Loading fastapi_solo/aio/dependencies.py
Loading fastapi_solo/utils/misc.py
Loading fastapi_solo/utils/db.py
Loading fastapi_solo/utils/inject.py
Loading fastapi_solo/utils/pagination.py
Loading fastapi_solo/utils/testing.py
Loading fastapi_solo/db/database.py
Loading fastapi_solo/db/queryable.py
Loading fastapi_solo/router/router.py
Loading fastapi_solo/router/solipsist.py


In [19]:
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

llm = ChatGroq(model="llama3-8b-8192", api_key="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")

prompt = ChatPromptTemplate.from_messages([
  ('system','You are my helpful code documentation assistant.'),
  ('human', "Answer to the user question using the given context.\n\nContext:\n{context}\n\nQuestion:\n{question}"),
])

chain = prompt | llm | StrOutputParser()

question = input('>')
context = "---\n".join([doc.page_content for doc in chroma.search(question, "similarity")])


print(chain.invoke({"context": context, "question": question}))


You can add a new filter to a model by using the `find_or_create` method. 

Here's an example:

```
user = User.find_or_create(find_by=["name"], name="Albert")
```

In this example, the `find_or_create` method will try to find a user with the name "Albert". If no user is found, it will create a new user with the name "Albert". 

If you want to add a new filter that is not one of the primary keys of the model, you can pass the `find_by` parameter to the `find_or_create` method. 

For example:

```
user = User.find_or_create(find_by=["email", "name"], email="john@example.com", name="John")
```

In this example, the `find_or_create` method will try to find a user with the email "john@example.com" and the name "John". If no user is found, it will create a new user with the email "john@example.com" and the name "John".
